In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
# generate X and beta that is fixed
num_samples = 100
X = np.random.normal(size=(num_samples,2))
X = np.hstack([np.ones((num_samples,1)), X])
beta = np.array([1,2,3]).T

In [3]:
# those that actually vary
num_sims = 10000
t_stats = np.empty(num_sims)
for i in range(num_sims):
    Y = X @ beta + np.random.normal(size=(num_samples,))
    XY = X.T @ Y
    XtX = X.T @ X
    XtXinv = np.linalg.inv(XtX)
    beta_hat = np.linalg.lstsq(XtX, XY)[0]
    pred = X @ beta_hat
    resid = Y - pred
    rss = (resid ** 2).sum()
    eta = np.array([0, 1, -2/3])
    eta_beta = np.dot(eta, beta_hat)
    t_stats[i] = eta_beta / np.sqrt(rss / (num_samples -3) * (eta @ np.linalg.inv(XtX) @ eta.T))

In [4]:
# check t_stats have variance approximately 1
t_stats.var()

np.float64(1.01776543344229)

In [5]:
# identical to the previous, but confidence interval
num_sims = 10000
confidence_intervals = np.empty((num_sims, 2))
for i in range(num_sims):
    Y = X @ beta + np.random.normal(size=(num_samples,))
    XY = X.T @ Y
    XtX = X.T @ X
    XtXinv = np.linalg.inv(XtX)
    beta_hat = np.linalg.lstsq(XtX, XY)[0]
    pred = X @ beta_hat
    resid = Y - pred
    rss = (resid ** 2).sum()
    eta = np.array([0, 1, -2/3])
    eta_beta = np.dot(eta, beta_hat)
    margin = np.sqrt(rss / (num_samples -3) * (eta @ np.linalg.inv(XtX) @ eta.T))
    t_975 = stats.t.ppf(0.975, num_samples-3)
    confidence_intervals[i] = eta_beta - t_975 * margin, eta_beta + t_975 * margin
coverage_probability = (confidence_intervals[:,0] * confidence_intervals[:,1] < 0).mean()
coverage_probability

np.float64(0.9477)